# RANDOM FOREST AND GBDT ON AMAZON REVIEWS DATASET

## about dataset:

Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1- Id<br>
2- ProductId - unique identifier for the product<br>
3- UserId - unqiue identifier for the user<br>
4- ProfileName<br>
5- HelpfulnessNumerator - number of users who found the review helpful<br>
6- HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not<br>
7- Score - rating between 1 and 5<br>
8- Time - timestamp for the review<br>
9- Summary - brief summary of the review<br>
10- Text - text of the review<br>


In [11]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer




con = sqlite3.connect('./amazon-fine-food-reviews/database.sqlite') 
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con)

In [12]:
s1= filtered_data.loc[filtered_data["Score"]>=4].sample(n=4500,random_state=1)
print(s1.shape)

s2= filtered_data.loc[filtered_data["Score"]<=2].sample(n=6500,random_state=127)
print(s2.shape)
data=s1
data=data.append(s2)
data.shape
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
data['Score'] = positiveNegative

(4500, 10)
(6500, 10)


In [13]:
sorted_data=data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final=final.drop_duplicates(subset={"UserId","ProfileName","Time"},keep='first',inplace=False)
final.shape
final['Score'].value_counts()

negative    5990
positive    4434
Name: Score, dtype: int64

In [14]:
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;    
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english') 

def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
i=0
str1=' '
final_string=[]
all_positive_words=[] 
all_negative_words=[] 
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    
    sent=cleanhtml(sent) 
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) 
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) 
                else:
                    continue
            else:
                continue 

    str1 = b" ".join(filtered_sentence)
    
    final_string.append(str1)
    i+=1

1
okay, it's freeze-dried liver cube-lets.  my common sense tells me that for a lot less than the price of this treat i could buy real liver, cube it and freeze it.  it would serve the same purpose for a lot less money, and it would not be over-processed.<br />that said, our dog would do back flips for this treat, so if you're made of money, go ahead.  you'll make your dog's day  :-)
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [15]:
final['CleanedText']=final_string
final.head(3) 


conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, flavor=None, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None,
             dtype=None)
final.shape

(10424, 11)

In [16]:
final=final.sort_values('Time')
x= np.array(final.iloc[:, 0:10])
y= np.array(final['Score'])


In [17]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import cross_validation

## Random Forest on Bag of words


In [28]:

count_vect = CountVectorizer()
final_bow = count_vect.fit_transform(x[:,9])


In [29]:
final_bow.get_shape()

(10424, 21944)

In [31]:
X_1, X_test, y_1, y_test = cross_validation.train_test_split(final_bow, y, test_size=0.3)

X_tr, X_cv, y_tr, y_cv = cross_validation.train_test_split(X_1, y_1, test_size=0.3)


In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_tr,y_tr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)
    


CV accuracy is 74%


In [22]:
cv_scores = []
baselearners=list(range(10,60,10))
for d in baselearners:
    clf = RandomForestClassifier(n_estimators=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)


The optimal estimators are 50.


In [34]:
clf = RandomForestClassifier(n_estimators=50)
clf = clf.fit(X_tr,y_tr)
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)
    


test accuracy is 78%


## GBDT on Bag Of Words

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
clf= GradientBoostingClassifier()
clf.fit(X_tr,y_tr)
pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)
    


CV accuracy is 79%


In [36]:
from sklearn.ensemble import GradientBoostingClassifier
cv_scores = []
lr=[0.1,0.3,0.5,0.7,1.0]
baselearners=list(range(100,1000,200))
depth=list(range(1,6,1))
for bl in baselearners:
    clf = GradientBoostingClassifier(n_estimators=bl)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)






The optimal estimators are 900.


In [20]:
for l in range(5):
    clf = GradientBoostingClassifier(learning_rate=lr[l])
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_learningrate = lr[MSE.index(min(MSE))]
print('\nThe optimal learning rate is %d.' % optimal_learningrate)




The optimal learning rate is 1.


In [19]:
for d in depth:
    clf = GradientBoostingClassifier(max_depth=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())
MSE = [1 - x for x in cv_scores]
optimal_depth = depth[MSE.index(min(MSE))]
print('\nThe optimal depth is %d.'% optimal_depth)



The optimal depth is 5.


In [21]:
clf = GradientBoostingClassifier(n_estimators=900,learning_rate=1.0,max_depth=5)
clf = clf.fit(X_tr,y_tr)
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 83%


In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[1570,  198],
       [ 324, 1036]], dtype=int64)

## Observation:
- the optimal base learners are 50 and 900 for RF and GBDT respectively.
- Based on Accuracy of both CV and Test data GBDT is performing slightly better than RF with Optimal learning rate 1 and depth of 5.

## Random Forest on TFIDF

In [9]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(x[:,9])
final_tf_idf.get_shape()
X_2, X_test1, y_2, y_test1 = cross_validation.train_test_split(final_tf_idf, y, test_size=0.3)

X_tr1, X_cv1, y_tr1, y_cv1 = cross_validation.train_test_split(X_2, y_2, test_size=0.3)


In [24]:
from sklearn.ensemble import RandomForestClassifier
cv_scores = []
baselearners=list(range(10,60,10))
for d in baselearners:
    clf = RandomForestClassifier(n_estimators=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)




The optimal estimators are 50.


In [26]:
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_tr,y_tr)

pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)
    


CV accuracy is 78%


In [27]:
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 79%


In [28]:
confusion_matrix(y_test,pred)

array([[1670,   98],
       [ 539,  821]], dtype=int64)

## GBDT on TFIDF

In [29]:
cv_scores = []
lr=[0.1,0.3,0.5,0.7,1.0]
baselearners=list(range(100,1000,200))
depth=list(range(1,6,1))
for bl in baselearners:
    clf = GradientBoostingClassifier(n_estimators=bl)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)

for l in range(5):
    clf = GradientBoostingClassifier(learning_rate=lr[l])
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_learningrate = lr[MSE.index(min(MSE))]
print('\nThe optimal learning rate is %d.' % optimal_learningrate)

for d in depth:
    clf = GradientBoostingClassifier(max_depth=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())
MSE = [1 - x for x in cv_scores]
optimal_depth = depth[MSE.index(min(MSE))]
print('\nThe optimal depth is %d.'% optimal_depth)



The optimal estimators are 900.

The optimal learning rate is 1.

The optimal depth is 5.


In [30]:
clf= GradientBoostingClassifier(n_estimators=900,learning_rate=1.0,max_depth=5)
clf.fit(X_tr,y_tr)
pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)



CV accuracy is 83%


In [31]:
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 82%


In [32]:
confusion_matrix(y_test,pred)

array([[1559,  209],
       [ 323, 1037]], dtype=int64)

## Observation :
- the optimal base learners are 50 and 900 for RF and GBDT respectively.
- based on the results the GBDT model is performing good with accuracies of over 80 in both test and Cv data with optimal learning rate of 1.0 and depth of 5.

## Random Forest on Avg_W2V

In [33]:
X_1, X_test, y_1, y_test = cross_validation.train_test_split(final['Text'], y, test_size=0.3)

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import gensim
i=0
list_of_sent=[]
for sent in X_1.values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)    
words = list(w2v_model.wv.vocab)
    
avg_w2v = [];
for sent in list_of_sent:
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    avg_w2v.append(sent_vec)
print(len(avg_w2v))
print(len(avg_w2v[0]))

7296
50


In [34]:

X_tr, X_cv, y_tr, y_cv = cross_validation.train_test_split(avg_w2v, y_1, test_size=0.3)

In [35]:
from sklearn.ensemble import RandomForestClassifier
cv_scores = []
baselearners=list(range(10,60,10))
for d in baselearners:
    clf = RandomForestClassifier(n_estimators=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)




The optimal estimators are 50.


In [36]:
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_tr,y_tr)

pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)
    


CV accuracy is 72%


In [38]:
i=0
list_of_sent=[]
for sent in X_test.values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)    
words = list(w2v_model.wv.vocab)
avg_w2v = [];
for sent in list_of_sent:
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    avg_w2v.append(sent_vec)
print(len(avg_w2v))
print(len(avg_w2v[0]))    

3128
50


In [40]:
pred = clf.predict(avg_w2v)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 45%


In [41]:
confusion_matrix(y_test,pred)

array([[ 419, 1378],
       [ 313, 1018]], dtype=int64)

## GBDT on Avg_W2V

In [44]:
cv_scores = []
lr=[0.1,0.3,0.5,0.7,1.0]
baselearners=list(range(100,1000,200))
depth=list(range(1,6,1))
for bl in baselearners:
    clf = GradientBoostingClassifier(n_estimators=bl)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)

for l in range(5):
    clf = GradientBoostingClassifier(learning_rate=lr[l])
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_learningrate = lr[MSE.index(min(MSE))]
print('\nThe optimal learning rate is %d.' % optimal_learningrate)




The optimal estimators are 300.

The optimal learning rate is 0.


In [70]:
cv_scores=[]
for d in range(1,6):
    clf = GradientBoostingClassifier(max_depth=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())
MSE = [1 - x for x in cv_scores]
optimal_depth = depth[MSE.index(min(MSE))]
print('\nThe optimal depth is %d.'% optimal_depth)



The optimal depth is 4.


In [74]:
clf= GradientBoostingClassifier(n_estimators=300,learning_rate=0.1,max_depth=4)
clf.fit(X_tr,y_tr)
pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)



CV accuracy is 73%


In [75]:
pred = clf.predict(avg_w2v)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 49%


In [76]:
confusion_matrix(y_test,pred)

array([[934, 863],
       [726, 605]], dtype=int64)

## Observation:
- here both the models perform almost same with similar accuracy rates just a little higher in GBDT's Test accuracy.
- the optimal estimators are 40,300 for RF and GBDT respectively and GBDT optimal depth as 4.

## Random Forest on TFIDF-W2V

In [37]:
X_1, X_test, y_1, y_test = cross_validation.train_test_split(x[:,9], y, test_size=0.3)

In [39]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(X_1)
print(final_tf_idf.get_shape())



(7296, 223920)


In [42]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import gensim

i=0
list_of_sent=[]
for sent in X_1:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)    
words = list(w2v_model.wv.vocab)
avg_w2v = [];
for sent in list_of_sent:
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    avg_w2v.append(sent_vec)
print(len(avg_w2v))
print(len(avg_w2v[0]))    

7296
50


In [43]:
tfidf_feat = tf_idf_vect.get_feature_names() 

tfidf_sent_vectors = []; 
row=0;
for sent in list_of_sent: 
    sent_vec = np.zeros(50) 
    weight_sum =0;
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1
     
    

In [44]:
X_tr, X_cv, y_tr, y_cv = cross_validation.train_test_split(tfidf_sent_vectors , y_1 , test_size=0.3)

In [45]:
from sklearn.ensemble import RandomForestClassifier
cv_scores = []
baselearners=list(range(10,60,10))
for d in baselearners:
    clf = RandomForestClassifier(n_estimators=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)




The optimal estimators are 40.


In [46]:
clf = RandomForestClassifier(n_estimators=40)
clf.fit(X_tr,y_tr)

pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)
    


CV accuracy is 68%


In [47]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(X_test)
print(final_tf_idf.get_shape())

i=0
list_of_sent=[]
for sent in X_test:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)    
words = list(w2v_model.wv.vocab)
avg_w2v = [];
for sent in list_of_sent:
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    avg_w2v.append(sent_vec)
print(len(avg_w2v))
print(len(avg_w2v[0]))    

(3128, 121035)
3128
50


In [48]:
tfidf_feat = tf_idf_vect.get_feature_names() 

tfidf_sent_vectors = []; 
row=0;
for sent in list_of_sent: 
    sent_vec = np.zeros(50) 
    weight_sum =0;
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1
     
    

In [49]:
pred = clf.predict(tfidf_sent_vectors)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 49%


In [50]:
confusion_matrix(y_test,pred)

array([[820, 976],
       [616, 716]], dtype=int64)

## GBDT on TFIDF-W2V

In [51]:
cv_scores = []
lr=[0.1,0.3,0.5,0.7,1.0]
baselearners=list(range(100,1000,200))
depth=list(range(1,6,1))
for bl in baselearners:
    clf = GradientBoostingClassifier(n_estimators=bl)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_estimators = baselearners[MSE.index(min(MSE))]
print('\nThe optimal estimators are %d.' % optimal_estimators)

for l in range(5):
    clf = GradientBoostingClassifier(learning_rate=lr[l])
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

MSE = [1 - x for x in cv_scores]

optimal_learningrate = lr[MSE.index(min(MSE))]
print('\nThe optimal learning rate is %d.' % optimal_learningrate)




The optimal estimators are 300.

The optimal learning rate is 0.


In [52]:
print(optimal_learningrate)

0.3


In [53]:
cv_scores=[]
for d in range(1,6):
    clf = GradientBoostingClassifier(max_depth=d)
    scores = cross_val_score(clf, X_tr, y_tr, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())
MSE = [1 - x for x in cv_scores]
optimal_depth = depth[MSE.index(min(MSE))]
print('\nThe optimal depth is %d.'% optimal_depth)



The optimal depth is 5.


In [54]:
clf= GradientBoostingClassifier(n_estimators=300,learning_rate=0.3,max_depth=5)
clf.fit(X_tr,y_tr)
pred = clf.predict(X_cv)
acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
print('\nCV accuracy is %d%%' %  acc)



CV accuracy is 69%


In [55]:
pred = clf.predict(tfidf_sent_vectors)
acc = accuracy_score(y_test, pred, normalize=True) * float(100)
print('\ntest accuracy is %d%%' %  acc)



test accuracy is 52%


In [56]:
confusion_matrix(y_test,pred)

array([[1085,  711],
       [ 789,  543]], dtype=int64)

## Observation:
- as seen above the optimal estimators or base learners are 40 and 300 for Random Forest and GBDT respectively.
- as compared with accuracies of both models the performance are almost identical with GBDT's Test accuracy just a little higher than random forest.

# CONCLUSION:



## RANDOM_FOREST:

In [63]:
from ipywidgets import *
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["VECTORIZER", "Optimal BaseLearners", "CV-accuracy", "Test Accuracy"]

x.add_row(["BOW", 50, 74, 78])
x.add_row(["TFIDF", 50, 78, 79])
x.add_row(["AVG-W2V", 50, 72, 45])
x.add_row(["TFIDF-W2V", 40, 68, 49])

print(x)

+------------+----------------------+-------------+---------------+
| VECTORIZER | Optimal BaseLearners | CV-accuracy | Test Accuracy |
+------------+----------------------+-------------+---------------+
|    BOW     |          50          |      74     |       78      |
|   TFIDF    |          50          |      78     |       79      |
|  AVG-W2V   |          50          |      72     |       45      |
| TFIDF-W2V  |          40          |      68     |       49      |
+------------+----------------------+-------------+---------------+


## GBDT:

In [65]:
x = PrettyTable()

x.field_names = ["VECTORIZER", "Optimal BaseLearners", "Optimal learningrate", "Optimal depth", "CV-accuracy", "Test Accuracy"]

x.add_row(["BOW", 900, 1, 5, 79, 83])
x.add_row(["TFIDF", 900, 1, 5, 83, 82])
x.add_row(["AVG-W2V", 300, 0.1, 4, 73, 49])
x.add_row(["TFIDF-W2V", 300, 0.3, 5, 69, 52])

print(x)

+------------+----------------------+----------------------+---------------+-------------+---------------+
| VECTORIZER | Optimal BaseLearners | Optimal learningrate | Optimal depth | CV-accuracy | Test Accuracy |
+------------+----------------------+----------------------+---------------+-------------+---------------+
|    BOW     |         900          |          1           |       5       |      79     |       83      |
|   TFIDF    |         900          |          1           |       5       |      83     |       82      |
|  AVG-W2V   |         300          |         0.1          |       4       |      73     |       49      |
| TFIDF-W2V  |         300          |         0.3          |       5       |      69     |       52      |
+------------+----------------------+----------------------+---------------+-------------+---------------+


## Key points:
- From the above results and Tables above we can state that our GBDT is performing better than RF model on the taken data sample.
- Overall from the taken dataset the Tfidf_GBDT model is performing better than other models and vectorizers combination.
- This may be true only in my case beacuse I have taken a sample of the reviews dataset so These conclusions are based on my sample dataset.